# Apache Kafkaのセットアップ

AKafka Connectをセットアップします。

# 事前チェック

Kafka Connectの構築を行うノードは「00-001-ノードの起動.ipynb」の Notebook で起動したVCノードである、または「00-002-既存ノードの登録」で準備したノードであることを想定しています。

それ以外のノードに対して環境構築を行う場合は、以下の条件を満たすように準備を行ってください。

1. 対象となるノードを Ansible で操作できる
1. Docker のセットアップが済んでいる
1. 構築に利用するツールがインストールされていること
  - rsync
  - docker-compose
1. ホスト名が名前解決できること

1., 2., 3. については、以下でチェックします。

In [ ]:
target = 'connector'

Ansibleで操作できることを確認します。

In [ ]:
!ansible {target} -m ping

In [ ]:
!ansible {target} -b -a 'whoami'

Dockerが利用できることを確認します。

In [ ]:
!ansible {target} -a 'docker info'

必要なコマンドがインストールされていることを確認します。

In [ ]:
!ansible {target} -a 'which rsync'

In [ ]:
!ansible {target} -a 'docker-compose version'

# パラメータの指定

Kafka Broker のサーバアドレスを指定してください。

> 「00-101-Apache Kafkaのセットアップ.ipynb」で構築したホスト名とポート番号の組からなるサーバアドレスのリストを指定してください。

In [ ]:
bootstrap_servers = [
    'broker-0:9092',
    'broker-1:9092',
    'broker-2:9092',
]

コネクタのグループIDを指定してください。

In [ ]:
group_id = 'connect-cluster-dp-01'

コネクタの設定値を保存するトピック名を指定してください。

In [ ]:
config_topic = 'connect-configs-dp-01'

In [ ]:
offset_topic = 'connect-offsets-dp-01'

In [ ]:
status_topic = 'connect-status-dp-01'

# Kafka Connect の実行

[Kafka Connect Configuration](https://docs.confluent.io/current/installation/docker/config-reference.html#kafka-connect-configuration)に示されている環境変数を設定します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

with TemporaryDirectory() as work_dir:
    docker_compose_yml = Path(work_dir) / 'docker-compose.yml'
    with docker_compose_yml.open(mode='w') as f:
        f.write('''version: '2.4'
services:
  connect:
    image: confluentinc/cp-kafka-connect
    environment:
      CONNECT_BOOTSTRAP_SERVERS: {% for sv in bootstrap_servers %}{%if not loop.first %},{% endif %}{{sv}}{% endfor %}

      CONNECT_GROUP_ID: {{group_id}}
      CONNECT_CONFIG_STORAGE_TOPIC: {{config_topic}}
      CONNECT_OFFSET_STORAGE_TOPIC: {{offset_topic}}
      CONNECT_STATUS_STORAGE_TOPIC: {{status_topic}}
      CONNECT_REST_PORT: 8083
      CONNECT_KEY_CONVERTER: org.apache.kafka.connect.json.JsonConverter
      CONNECT_VALUE_CONVERTER: org.apache.kafka.connect.json.JsonConverter
      CONNECT_INTERNAL_KEY_CONVERTER: org.apache.kafka.connect.json.JsonConverter
      CONNECT_INTERNAL_VALUE_CONVERTER: org.apache.kafka.connect.json.JsonConverter
      CONNECT_PLUGIN_PATH: /usr/share/java
{% if replication_factor is defined %}
      CONNECT_CONFIG_STORAGE_REPLICATION_FACTOR: "{{replication_factor}}"
      CONNECT_OFFSET_STORAGE_REPLICATION_FACTOR: "{{replication_factor}}"
      CONNECT_STATUS_STORAGE_REPLICATION_FACTOR: "{{replication_factor}}"
{% endif %}
      CONNECT_REST_ADVERTISED_HOST_NAME: {{inventory_hostname_short}}
    hostname: {{inventory_hostname_short}}
    env_file: .env
    network_mode: 'host'
    init: true
    restart: always
''')
    !ansible {target} -m file -a 'path=kafka-connect state=directory'
    params = {
        'bootstrap_servers': bootstrap_servers,
        'group_id': group_id,
        'config_topic': config_topic,
        'offset_topic': offset_topic,
        'status_topic': status_topic,
    }
    if len(bootstrap_servers) < 3:
        params['replication_factor'] = len(bootstrap_servers)
    params_json = Path(work_dir) / 'params.json'
    with params_json.open(mode='w') as f:
        json.dump(params, f)
    !ansible {target} -m template -e @{params_json} \
        -a 'src={docker_compose_yml} dest=kafka-connect/'
    !ansible {target} -m file -a 'path=kafka-connect/.env state=touch'        

コンテナを実行します。

In [ ]:
!ansible {target} -a 'chdir=kafka-connect docker-compose up -d'

コンテナの状態を確認します。

In [ ]:
!ansible {target} -a 'chdir=kafka-connect docker-compose ps'